In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import xarray as xr; import time as time
import matplotlib.colors as colors
import matplotlib.lines as mlines

dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
folder_path = dir + 'Figures/plots/'
import os; os.makedirs(folder_path, exist_ok=True)

# Importing Model Data
check=False
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'

# # dx = 1 km; Np = 1M; Nt = 5 min
# data=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_1km_5min.nc', decode_timedelta=True) #***
# parcel=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_pdata_1km_5min_1e6.nc', decode_timedelta=True) #***
# res='1km';t_res='5min'
# Np_str='1e6'

# # dx = 1km; Np = 50M
# #Importing Model Data
# dir2='/home/air673/koa_scratch/'
# data=xr.open_dataset(dir2+'cm1out_1km_1min.nc', decode_timedelta=True) #***
# parcel=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_50M.nc', decode_timedelta=True) #***
# res='1km'; t_res='1min'; Np_str='50e6'

# dx = 1km; Np = 50M; Nz = 95
#Importing Model Data
dir2='/home/air673/koa_scratch/'
data=xr.open_dataset(dir2+'cm1out_1km_1min_95nz.nc', decode_timedelta=True) #***
parcel=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_95nz.nc', decode_timedelta=True) #***
res='1km'; t_res='1min_95nz'; Np_str='50e6'

In [ ]:
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
whereCL=xr.open_dataset(dir+'Project_Algorithms/whereCL_4_0622.nc')
def get_3dtime_data(data,varname,tlev):
    cloud_var=data[varname].isel(time=tlev).values
    return cloud_var
def cd2d(f,dx,dy): #size not compatible, cant calculate adjacent gradient
    ddx = (
            f[:,:, 1:  ]
            -
            f[:,:, 0:-1]
        ) / (
        2 * dx
    )
    
    ddy = (
        f[:,1:, :]
        -
        f[:,0:-1, :]
    ) / (
        2 * dy
    )
    
    return ddx, ddy

# #print longnames
# {var: data[var].attrs['long_name'] for var in data.variables if 'long_name' in data[var].attrs}

In [ ]:
#Import PlottingFunctions
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
path=dir2+'../Functions/'
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions


# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

In [ ]:
#Grid Stretching
######################################################################################################################################################
from matplotlib.gridspec import GridSpec
zfs = data['zf'].values;zhs = data['zh'].values
zfs_diff = np.diff(zfs) 
fig = plt.figure(figsize=(10, 4))
gs = GridSpec(1, 2, figure=fig)
ax0 = fig.add_subplot(gs[0, 0]); ax1 = fig.add_subplot(gs[0, 1]) 

ax0.set_xlabel('z grid');ax0.set_ylabel('z (km)')
ax1.set_xlabel('dz');ax1.set_ylabel('z (km)')
ax0.plot(np.arange(len(zfs)), zfs);ax0.set_title('z level heights (kms)')
ax1.plot(zfs_diff*1000,zhs);ax1.set_title('z level height diffs (m)')

In [ ]:
#Domain Configuration (1km)
######################################################################################################################################################
#ocean_fraction
# ocean_fraction=3/8
# ocean_fraction=2.5/8
ocean_fraction=2/8

#Splitting Domain
# bottom=-50; top=50;
bottom=-100; top=100;
# left=-256;right=256;center=0
left=-256;right=256;center=left+((right-left)*ocean_fraction)


###############################################################

#Setting Up Plot
channel_aspect_ratio = 5; dpi=300
plt.figure(figsize=(10, 10/channel_aspect_ratio));
plt.xlabel('x (km)')
plt.ylabel('y (km)')
plt.title(f'Domain Configuration ({ocean_fraction})')
plt.xlim(left, right); plt.xticks(range(left, right+1, right));
plt.ylim(bottom, top); plt.yticks(range(bottom, top+1, top));

#Plotting
seacolor=(0, 89/255, 179/255)
landcolor=(139/255, 69/255, 19/255) 
plt.text((left+center)/2,0,'Sea'); 

plt.axvspan(left, center, color=seacolor, alpha=0.3) 
plt.text((right+center)/2-10,0,'Land'); 
plt.axvspan(center, right, color=landcolor, alpha=0.3) 

#Center Line
plt.axvline(x=center, color='black',linewidth=3);

# plt.axvline(x=30, color='black',linewidth=3,linestyle='dashed');
# # plt.axvline(x=0, color='black',linewidth=3,linestyle='dashed');

# plt.savefig(os.path.join(folder_path, f"domain_config.jpg"),dpi=dpi) 
plt.savefig(os.path.join(folder_path, f"domain_config_{ocean_fraction}.jpg"),dpi=dpi) 


In [ ]:
load_dir = '/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/Project_Algorithms/Tracking_Algorithms/'
whereCL_ALL_CLS=xr.open_dataset(load_dir+f'whereCL_{res}_ALL_CLS.nc')['maxconv_x'].data
whereCL_ONLY_SBZS=xr.open_dataset(load_dir+f'whereCL_{res}_ONLY_SBZS.nc')['maxconv_x'].data

def get_conv(t):
    import h5py
    # print('calculating convergence and taking mean')
    dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
    file_path = dir2 + 'Variable_Calculation/' + 'Convergence' + f'_{res}_{Np_str}' + '.h5'
    with h5py.File(file_path, 'r') as f:
        Conv = f['HMC'][t]
    return Conv

In [ ]:
def CL_plotting(t, ax, ONLY_SBZ):
    if np.mod(t,10)==0: print(f'current time step: {t}/{len(data["time"])}')

    # Taking Convergence of current timestep
    dx = np.round(data['xf'][1] - data['xf'][0], 2).item()
    dy = dx  # gets the dx, dy in meters (0.25m in this case)
    conv = get_conv(t)

    # Plotting horizontal layer at ~0.28 km
    zlev = 3  # (zf level ~0.28 km)
    # channel_aspect_ratio = 5
    # figwidth = 20
    # fig, ax = plt.subplots(figsize=(figwidth, figwidth / channel_aspect_ratio))  # use ax for the plot

    # print(conv[zlev].shape)
    contour = ax.contourf(conv[zlev] * 1000, levels=40)#,extent=[0,512,0,200])  # contour plot
    # print(ax.get_yticks())
    # print(ax.get_xticks())

    colorbar = plt.colorbar(contour, label=f'{dx * 1000:.0f}' + r'$\ s^{-1}$')

    # Gets time for Plot Label
    ([days, hours, mins], combo) = get_time(data,t + index_adjust,(0,6,0))
    value = data['zf'][zlev].values * 1000
    lang = 'With Tracked Local Y-Maxima of Convergence Overlayed'

    ax.set_title(f'Horizontal Convergence at t = {t + index_adjust} = {days}:{hours}:{mins}, z = {value:.1f} m\n ' + lang)
    ax.set_xlabel('x (km)')
    ax.set_ylabel('y (km)')

    # Adding tracked location scatter
    zlev = 3
    conv_z = conv[zlev, :, :]  # only look at 3rd level of convergence field
    maxconv_x = np.array([], dtype='int')
    
    for yind in range(0, len(data['yh'])):  # plot maximums for each row
        if ONLY_SBZ == False:
            local_maxes = whereCL_ALL_CLS[t, 3, yind]
            local_maxes = local_maxes[local_maxes != -1]
        elif ONLY_SBZ == True:
            local_maxes = whereCL_ONLY_SBZS[t, 3, yind]
            local_maxes = local_maxes[local_maxes != -1]
        
        ax.scatter(local_maxes, np.array([yind] * len(local_maxes)), color='red', s=1)

    # Plotting coastline
    ax.axvline(x=len(data['xf']) * ocean_fraction, color='black', linewidth=1.5, label='Coastline')
    ax.legend(loc='upper left')



    # Fixing tick labels
    fix_tick_labels([ax], data, tick_dim='x', data_dim='x', d_xtick=32, d_ytick=20, cell_loc='center', round=1, meters=False)  # apply fix_tick_labels on x-axis
    fix_tick_labels([ax], data, tick_dim='y', data_dim='y', d_xtick=32, d_ytick=20, cell_loc='center', round=1, meters=False)  # apply fix_tick_labels on y-axis

# Call the plotting function with the desired time step
t = 100


# Example call
t = 100
channel_aspect_ratio = 5
figwidth = 20
fig, axes = plt.subplots(nrows=1,ncols=1,figsize=(figwidth, figwidth / channel_aspect_ratio), dpi=dpi)
CL_plotting(t, axes, ONLY_SBZ=False)

# # Save figure
plt.savefig(os.path.join(folder_path, f"convergence.jpg"),dpi=dpi) 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import matplotlib.colors as colors
import matplotlib.lines as mlines

def plot_alongcoast_vertical_u(t, data, ocean_fraction, ax, maxlev=5, dpi=300, index_adjust=0):
    print(f'Plotting Along-coast Vertical U for time step {t}/{len(data["time"])}')

    # Extract data
    alongcoast_u = data['uinterp'].isel(time=t, zh=slice(0, np.where(data['zh'] < maxlev)[0][-1])).mean(dim='yh').data
    alongcoast_w = data['winterp'].isel(time=t, zh=slice(0, np.where(data['zh'] < maxlev)[0][-1])).mean(dim='yh').data
    alongcoast_w[alongcoast_w < 0.1] = 0  # Thresholding small values
    alongcoast_qv = data['qv'].isel(time=t, zh=slice(0, np.where(data['zh'] < maxlev)[0][-1])).mean(dim='yh').data

    # Normalize colormap
    n_levels = 15
    vmin = -np.max(abs(alongcoast_u))
    vmax = np.max(abs(alongcoast_u))
    levels = np.linspace(vmin, vmax, n_levels)
    norm = colors.BoundaryNorm(boundaries=levels, ncolors=n_levels)
    cmap = plt.get_cmap('RdBu_r', n_levels)

    # Contour plot for U
    contour_u = ax.contourf(alongcoast_u, cmap=cmap, norm=norm, levels=levels)
    cbar = plt.colorbar(contour_u, ax=ax, pad=0)
    cbar.set_label(r'$m\ s^{-1}$')

    ax.grid(True, linestyle=':', color='black')

    # Contour for Water Vapor Mixing Ratio (qv) and Vertical Velocity (w)
    contour_qv = ax.contour(alongcoast_qv * 1000, linewidths=1, colors='blue', alpha=0.75)
    ax.clabel(contour_qv, inline=True, fontsize=10, fmt='%1.1f', inline_spacing=5)

    contour_w = ax.contour(alongcoast_w, linestyles='dashed', linewidths=1, colors='black', alpha=0.75)

    # Plot coastline
    ax.axvline(x=len(data['xh']) * ocean_fraction, color='black', linewidth=3, label='Coast')

    # Add land and sea colors
    seacolor = (0, 89 / 255, 179 / 255)
    landcolor = (139 / 255, 69 / 255, 19 / 255)
    ax.axhline(y=0, xmin=0, xmax=ocean_fraction, color=seacolor, linewidth=10)
    ax.axhline(y=0, xmin=ocean_fraction, xmax=1, color=landcolor, linewidth=10)

    # Get time information for labeling
    ([days, hours, mins], _) = get_time(data, t + index_adjust, (0, 6, 0))

    ax.set_title(f'Along-coast Averaged Vertical Plot of Zonal Velocity at t={t+index_adjust}={days}:{hours}:{mins}')

    # Legend
    handle_u = mlines.Line2D([], [], color='black', lw=3, label='Coast')
    handle_qv = mlines.Line2D([], [], color='blue', label='qv Contours')  
    handle_w = mlines.Line2D([], [], color='black', linestyle='dashed', label='w Contours')  
    ax.legend(handles=[handle_qv, handle_w, handle_u], loc='upper left')

    ax.set_xlabel('x (km)')
    ax.set_ylabel('z (m)')

    #####
    #OLD
    # # Fix y-ticks properly for the given axis
    # ticks = ax.get_yticks()  # Get y-ticks for the specific subplot
    # valid_ticks = ticks[(ticks >= 0) & (ticks < len(data['zh']))]  # Ensure valid index range
    # if len(valid_ticks) > 0:  # Avoid indexing errors
    #     newticks = np.round(data['zh'].isel(zh=valid_ticks.astype(int)).values, 1)
    #     ax.set_yticks(valid_ticks)
    #     ax.set_yticklabels(newticks)
    #####

    #Fix x-ticks
    fix_tick_labels([ax], data, data_dim='x', tick_axis='x', d_xtick=32, d_ytick=20, cell_loc='center',round=1,meters=False)  # apply 
    
    #Fix y-ticks
    fix_tick_labels([ax], data, data_dim='z', tick_axis='y', d_xtick=32, d_ytick=8, cell_loc='center',round=1,meters=True)  # apply 



# Example call
t = 500
channel_aspect_ratio = 5
figwidth = 25
fig, axes = plt.subplots(nrows=1,ncols=1,figsize=(figwidth, figwidth / channel_aspect_ratio), dpi=dpi)
plot_alongcoast_vertical_u(t, data, ocean_fraction, ax=axes, index_adjust=0, dpi=300, maxlev=5) #if there are more than 1 change axes to axes[]

# Save figure
save_path = os.path.join(folder_path, "verticalu.jpg")
plt.savefig(save_path, dpi=dpi)
print(f"Figure saved at {save_path}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import matplotlib.colors as colors

def plot_vertical_pressure_perturbation(t, data, ocean_fraction, ax, index_adjust=0, dpi=300, maxlev=5):
    print(f'Plotting along-coast vertical pressure perturbation for time step {t}/{len(data["time"])}')

    # channel_aspect_ratio = 5
    # figwidth = 25
    # fig, ax = plt.subplots(figsize=(figwidth, figwidth / channel_aspect_ratio), dpi=dpi)
    
    # Extract pressure perturbation data and average along 'yh'
    alongcoast = data['prspert'].isel(time=t, zh=slice(0, np.where(data['zh'] < maxlev)[0][-1])).mean(dim='yh')
    
    # Normalize with balanced vmin and vmax
    n_levels = 15
    vmin = -np.max(abs(alongcoast))
    vmax = np.max(abs(alongcoast))
    levels = np.linspace(vmin, vmax, n_levels)
    norm = colors.BoundaryNorm(boundaries=levels, ncolors=n_levels)
    cmap = plt.get_cmap('RdBu_r', n_levels)
    
    # Contour plot
    contour = ax.contourf(alongcoast, cmap=cmap, norm=norm, levels=levels)
    cbar = plt.colorbar(contour, ax=ax, pad=0)
    # cbar.set_label(r'$Pa$')
    cbar.set_label(r'$\mathrm{Pa}^\prime$')
    ax.grid(True, linestyle=':', color='black')
    
    # Plot coastline and land/sea division
    ax.axvline(x=len(data['xh']) * ocean_fraction, color='black', linewidth=3, label='Coast')
    seacolor = (0, 89/255, 179/255)
    landcolor = (139/255, 69/255, 19/255)
    ax.axhline(y=0, xmin=0, xmax=ocean_fraction, color=seacolor, linewidth=10)
    ax.axhline(y=0, xmin=ocean_fraction, xmax=1, color=landcolor, linewidth=10)
    
    # Get time information for labeling
    ([days, hours, mins], _) = get_time(data, t + index_adjust, (0, 6, 0))
    ax.set_title(f'Along-coast Averaged Vertical Plot of Pressure Perturbation at t={t+index_adjust}={days}:{hours}:{mins}')
    ax.legend(loc='upper left')
    ax.set_xlabel('x (km)')
    ax.set_ylabel('z (m)')


    #Fix x-ticks
    fix_tick_labels([ax], data, data_dim='x', tick_axis='x', d_xtick=32, d_ytick=20, cell_loc='center',round=1,meters=False)  # apply 
    
    #Fix y-ticks
    fix_tick_labels([ax], data, data_dim='z', tick_axis='y', d_xtick=32, d_ytick=8, cell_loc='center',round=1,meters=True)  # apply 

# Example call
t = 500
channel_aspect_ratio = 5
figwidth = 25
fig, axes = plt.subplots(nrows=1,ncols=1,figsize=(figwidth, figwidth / channel_aspect_ratio), dpi=dpi)
plot_vertical_pressure_perturbation(t, data, ocean_fraction, ax=axes, index_adjust=0, dpi=300, maxlev=5) #if there are more than 1 change axes to axes[]

# Save figure
# plt.close(fig)
# print(f"Figure saved at {save_path}")
save_path = os.path.join(folder_path, "verticalprspert.jpg")
plt.savefig(save_path, dpi=dpi)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import matplotlib.colors as colors

def plot_alongcoast_water_vapor(t, data, ocean_fraction, ax, maxlev=5, index_adjust=0, dpi=300):
    print(f'Plotting Along-coast Water Vapor Mixing Ratio for time step {t}/{len(data["time"])}')
    pad=0
    
    alongcoast = data['qv'].isel(time=t, zh=slice(0, np.where(data['zh'] < maxlev)[0][-1])).mean(dim='yh') * 1000
    
    c = ax.contourf(alongcoast, cmap='Blues', levels=15)
    cbar = fig.colorbar(c, ax=ax, pad=pad)
    cbar.set_label(r'$kg\ kg^{-1}$')
    ax.grid(True, linestyle=':', color='black')

    # Plot coastline
    ax.axvline(x=len(data['xh']) * ocean_fraction, color='black', linewidth=3, label='Coast')

    # Add land and sea colors
    seacolor = (0, 89 / 255, 179 / 255)
    landcolor = (139 / 255, 69 / 255, 19 / 255)
    ax.axhline(y=0, xmin=0, xmax=ocean_fraction, color=seacolor, linewidth=10)
    ax.axhline(y=0, xmin=ocean_fraction, xmax=1, color=landcolor, linewidth=10)

    # Get time information for labeling
    ([days, hours, mins], _) = get_time(data, t + index_adjust, (0, 6, 0))

    ax.set_title(f'Along-coast Vertical Plot of Water Vapor Mixing Ratio at t={t+index_adjust}={days}:{hours}:{mins}')
    ax.legend(loc='upper left')
    ax.set_xlabel('x (km)')
    ax.set_ylabel('z (m)')

    #Fix x-ticks
    fix_tick_labels([ax], data, data_dim='x', tick_axis='x', d_xtick=32, d_ytick=20, cell_loc='center',round=1,meters=False)  # apply 
    
    #Fix y-ticks
    fix_tick_labels([ax], data, data_dim='z', tick_axis='y', d_xtick=32, d_ytick=8, cell_loc='center',round=1,meters=True)  # apply 


# Example call
t = 500
channel_aspect_ratio = 5
figwidth = 25
fig, axes = plt.subplots(nrows=1,ncols=1,figsize=(figwidth, figwidth / channel_aspect_ratio), dpi=dpi)
plot_alongcoast_water_vapor(t, data, ocean_fraction, axes)

# Save figure
save_path = os.path.join(folder_path, "verticalwatervapor.jpg")
plt.savefig(save_path, dpi=dpi)
# plt.close(fig)
# print(f"Figure saved at {save_path}")

In [ ]:
def plot_horizontal_wind_velocity(t, data, ocean_fraction, index_adjust=0, dpi=300):

    print(f'Plotting horizontal wind and velocity for time step {t}/{len(data["time"])}')

    zlev = 3  # Vertical level
    channel_aspect_ratio = 5
    fig, axes = plt.subplots(2, 1, figsize=(25, 50 / channel_aspect_ratio), dpi=dpi)

    # List of variables to plot
    variables = ['winterp', 'uinterp']
    titles = ['Horizontal Plot of Vertical Velocity', 'Horizontal Plot of Zonal Wind']
    color_labels = [r'$m\ s^{-1}$', r'$m\ s^{-1}$']

    for i, (var, title, c_label) in enumerate(zip(variables, titles, color_labels)):
        ax = axes[i]
        field = data[var].isel(time=t, zh=zlev)

        # Normalize with balanced vmin and vmax
        n_levels = 15
        vmin = -np.max(abs(field))
        vmax = np.max(abs(field))
        levels = np.linspace(vmin, vmax, n_levels)
        norm = colors.BoundaryNorm(boundaries=levels, ncolors=n_levels)
        cmap = plt.get_cmap('bwr', n_levels)

        contour = ax.contourf(field, cmap=cmap, norm=norm, levels=levels)
        plt.colorbar(contour, ax=ax, label=c_label, pad=0)

        # Plot coastline
        ax.axvline(x=len(data['xh']) * ocean_fraction, color='black', linewidth=3, label='Coast')

        # Get time information for labeling
        ([days, hours, mins], _) = get_time(data, t + index_adjust, (0, 6, 0))
        value = data['zh'][zlev].values

        # Set titles and labels
        ax.set_title(f'{title} at t={t+index_adjust}={days}:{hours}:{mins}, z={value:.2f} km' if i == 0 else title)
        ax.legend(loc='upper left')
        ax.set_xlabel('x (km)')
        ax.set_ylabel('y (km)')


        #Fix x-ticks
        fix_tick_labels([ax], data, data_dim='x', tick_axis='x', d_xtick=32, d_ytick=20, cell_loc='center',round=1,meters=False)  # apply 
        
        #Fix y-ticks
        fix_tick_labels([ax], data, data_dim='y', tick_axis='y', d_xtick=32, d_ytick=20, cell_loc='center',round=1,meters=False)  # apply 

    # Save figure
    save_path = os.path.join(folder_path, "horizontalw-u.jpg")
    plt.savefig(save_path, dpi=dpi)
    # plt.close(fig)
    # print(f"Figure saved at {save_path}")

# Example call
t = 500
plot_horizontal_wind_velocity(t, data, ocean_fraction,  index_adjust=0, dpi=300)

In [ ]:
#*#*#*#*#*#*#*#*#*#*#

In [ ]:
#TROUBLE USING DASK 
data2 = xr.open_dataset(
    dir2 + 'cm1out_1km_1min_95nz.nc',
    chunks={'time': 100, 'zh': 32, 'yh': 100, 'xh': 128}
)
import dask.array as da
from dask.distributed import Client, LocalCluster
from dask.diagnostics import ProgressBar
cluster = LocalCluster(n_workers=4, threads_per_worker=1)
client = Client(cluster)

with ProgressBar():
    qcqi_tz = (data2['qc'] + data2['qi']).mean(dim=('yh', 'xh')).load()

###################################
import dask.array as da
compute_mean=da.mean(data2['qc'].data+data2['qi'].data,axis=(-2, -1))
with ProgressBar():
    qcqi_tz=compute_mean.compute()

In [ ]:
#CURRENTLY PLOTTED BY FIRST RUN

import numpy as np
import matplotlib.pyplot as plt
import os
import matplotlib.colors as colors

def plot_horiz_avg_qcqi(data, ocean_fraction, ax, maxlev=5, dpi=300):
    pad=0

    ######
    #NO-DASK VERSION
    # qcqi_tz = (data['qc'] + data['qi']).mean(dim=('yh', 'xh'))
    
    w_tz=data['winterp'].data; w_tz[w_tz<0]=0; w_tz=np.mean(w_tz,axis=(2,3))
    
    c = ax.contourf(qcqi_tz.T, cmap='Blues', levels=5)
    cbar = fig.colorbar(c, ax=ax, pad=pad)
    cbar.set_label(r'$kg\ kg^{-1}$')
    ax.grid(True, linestyle=':', color='black')

    ax.contour(w_tz.T, levels=15, colors='black', linestyles='--', alpha=0.3)

    ax.set_title(f'Horizontal Average of qc+qi (color contours), w (line contours)')
    ax.set_xlabel('x (km)')
    ax.set_ylabel('z (m)')

    #Fix x-ticks
    fix_tick_labels([ax], data, data_dim='x', tick_axis='x', d_xtick=32, d_ytick=20, cell_loc='center',round=1,meters=False)  # apply 
    
    #Fix y-ticks
    fix_tick_labels([ax], data, data_dim='z', tick_axis='y', d_xtick=32, d_ytick=4, cell_loc='center',round=1,meters=True)  # apply 


# Example call
channel_aspect_ratio = 5
figwidth = 25
fig, axes = plt.subplots(nrows=1,ncols=1,figsize=(figwidth, figwidth / channel_aspect_ratio), dpi=dpi)
plot_horiz_avg_qcqi(data, ocean_fraction, axes)

# Save figure
save_path = os.path.join(folder_path, "horiz_avg_qcqi.jpg")
plt.savefig(save_path, dpi=dpi)
# plt.close(fig)
# print(f"Figure saved at {save_path}")

In [ ]:
################################################################################################################################################

In [ ]:
## Converts all figures to PDF
######################################################################################################################################################
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from PIL import Image
import os

def jpg_to_pdf(input_folder, output_pdf):
    # Get a list of all JPG files in the input folder
    jpg_files = [file for file in os.listdir(input_folder) if file.endswith('.jpg')]
    jpg_files = ['domain_config.jpg','convergence.jpg','verticalu.jpg',
                 'verticaltheta.jpg','verticalprspert.jpg','verticaltheta.jpg',
                 'verticalwatervapor.jpg','horizontalw-u.jpg'] #***
    
    # Create a PDF canvas
    c = canvas.Canvas(output_pdf, pagesize=letter)

    # Loop through each JPG file and add it to the PDF
    for jpg_file in jpg_files:
        # Open the JPG image using PIL
        img = Image.open(os.path.join(input_folder, jpg_file))

        # Calculate the aspect ratio to maintain image proportions
        width, height = img.size
        aspect_ratio = width / height

        # Add the image to the PDF
        c.setPageSize((width, height))
        c.drawInlineImage(os.path.join(input_folder, jpg_file), 0, 0, width=width, height=height)

        # Add a new page for the next image
        c.showPage()

    # Save the PDF
    c.save()

# Example usage:
input_folder = folder_path
output_pdf = folder_path + f'figures_062217_{res}.pdf'
jpg_to_pdf(input_folder, output_pdf)

In [ ]:
################################################################################################################################################

In [ ]:
# import matplotlib.pyplot as plt
# import matplotlib.animation as animation
# from matplotlib.animation import FuncAnimation
# import os

# channel_aspect_ratio = 5
# dpi=300;figwidth=25
# fig=plt.figure(figsize=(figwidth, figwidth/channel_aspect_ratio),dpi=dpi) # Create figure inside frame function

# def frame(t):
#     print(f'time = {t}')
#     plt.clf()
#     maxlev=3 #225-300
#     pad=0;

    
#     alongcoast=data['qv'].isel(time=t,zh=slice(0,np.where(data['zh']<maxlev)[0][-1])).mean(dim='yh')*1000
#     plt.contourf(alongcoast,cmap='Blues',levels=15)
#     cbar=plt.colorbar(pad=pad); cbar.set_label(r'$kg\ kg^{-1}$')
#     plt.grid(True, linestyle=':', color='black')
    
#     plt.axvline(x=data['xh'][-3].values, color='black',linewidth=3) #plotting coast
#     plt.axvline(x=data['xf'][-1].astype(int).item()+whereCL['maxconv_x'].isel(time=t,z=2).mean().values, color='green',linewidth=3) #plotting CL
#     seacolor=(0, 89/255, 179/255); landcolor=(139/255, 69/255, 19/255) 
#     plt.axhline(y=0, xmin=0, xmax=0.5, color=seacolor,linewidth=10);
#     plt.axhline(y=0, xmin=0.5, xmax=1, color=landcolor,linewidth=10);
    
#     plt.title('Along-coast Vertical Plot of Water Vapor Mixing Ratio')
#     plt.legend(('Coastline','Convergence Line'),loc='upper left')
#     plt.xlabel('x (km)')
#     plt.ylabel('z (km)')
#     #####################################################################
#     [days,hours,mins]=get_time(t)
#     plt.text(len(data['xf'])*0.9,7.4,f't={days}:{hours}:{mins}',fontsize=20) 
#     #####################################################################

    
#     #####################################################################
#     ticks, _ = plt.yticks()
#     newticks=np.round(data['zh'][ticks.astype(int)].values,1)
#     plt.yticks(ticks=ticks, labels=newticks);
    
#     plt.xticks(np.arange(0, len(data['xh']), 64));
#     ticks, _ = plt.xticks()
#     newticks=ticks-data['xf'][-1].values; newticks=newticks.astype(int)
#     plt.xticks(ticks=ticks, labels=newticks);

# start,end=225,301
# animation = FuncAnimation(fig, frame, frames=range(start, end+1), repeat=False)
# animation.save(os.path.join(folder_path, "verticalwatervapor.gif"), writer='Pillow', fps=10)

In [ ]:
# # #Comparing Vertical Velocity and Moisture around CL for single timesteps

# # find left and right of CL axis 
# import math
# kms=np.argmax(data['xh'].values-data['xh'][0].values >= 1) #finds how many x grids is 1 km
# def mirror(lst):
#     lst_mirror=lst[::-1]
#     return lst_mirror
# def make_xy(n):
#     dx=data['xh'].values[1]-data['xh'][0].values;dy=dx
#     #x
#     lst0=[-n]*n
#     lst1=list(np.arange(-(2*n-1)/2,((2*n-1)/2)+1,1))
#     lst2=[n]*(2*n)
#     lst3=[-n]*n
#     x=lst0+lst1+lst2+mirror(lst1)+lst3
#     x=np.array([ind*dx for ind in x])
#     #y
#     y=[]
#     arr1 = list(np.arange(2*n, (8*n)))
#     arr2 = list(np.arange(0, (2*n-1)+1))
#     rearange = arr1+arr2
#     y=[x[ind] for ind in rearange]
#     y=np.array([ind*dy for ind in y])
#     return x, y
# def make_inds_xy(n):
#     #inds_x
#     lst0=[-n]*n
#     lst1=list(np.arange(-n+1,(n-1)+1,1))
#     lst2=[n]*(2*n+1)
#     lst3=[-n]*(n+1)
#     inds_x=lst0+lst1+lst2+mirror(lst1)+lst3
#     inds_x=np.array(inds_x)
#     #inds_y
#     arr1 = list(np.arange(2*n, (8*n)))
#     arr2 = list(np.arange(0, (2*n-1)+1))
#     rearange = arr1+arr2
#     inds_y=np.array([inds_x[ind] for ind in rearange])
#     return inds_x, inds_y

# def leftright(u,v): #TESTING
#     #(0) getting wind angle
#     theta=math.atan2(v,u) #returns wind angle between -180 and 180
#     # print(f'theta is {math.degrees(theta)} deg')
    
#     ns=2*kms #2 kms around CL
#     #(1) starting for loop
#     xs,ys=[],[]
#     dx=data['xh'].values[1]-data['xh'][0].values;dy=dx
#     for n in range(1,(ns)+1):
#         #(2) make x and y
#         [x,y]=make_xy(n)
#         #(3) make indx and indy
#         [inds_x,inds_y]=make_inds_xy(n)
#         #(4) find tau
#         tau = np.arctan2(y, x)
#         #(5) use search sorted to find which range theta falls in 
#         ind=(8*n-2)-(np.searchsorted(mirror(tau),theta)-1)
#         #(6) get x and y inds
#         if np.degrees(theta) in [-180,180]: #needed since atan2 function discontinuity
#             xs.append(-n);ys.append(0)
#         else:
#             xs.append(inds_x[ind]);ys.append(inds_y[ind]) 
#         # print([inds_x[ind],inds_y[ind]])
#     xs=[-ind for ind in mirror(xs)]+[0]+xs
#     ys=[-ind for ind in mirror(ys)]+[0]+ys
#     # print(xs,ys)
#     return xs,ys
# ############################################################################################################
# def lrdata(t,var):
#     zlev=3
#     n=2*kms
#     result=np.zeros((2*n+1))
#     for ylev in range(len(data['yh'])):
#         print(f'y level: {ylev}')
#         CL=whereCL['maxconv_x'].isel(time=t,y=ylev,z=zlev).values
#         xlev=np.where(CL!=-1)[0]
#         for xlev in xlev:
#             u=data['u'].isel(time=t,yh=ylev,zh=zlev,xf=xlev).values
#             v=data['v'].isel(time=t,yf=ylev ,zh=zlev,xh=xlev).values
#             [xs,ys]=leftright(u,v);  
            
#             xrange=[ind+xlev for ind in xs]
#             yrange=[np.mod(ind+ylev,len(data['yh'])) for ind in ys]
    
#             n=2*kms
#             lr = np.zeros(2*n+1)
#             for ind in range(len(xrange)):
#                 if var=='w':
#                     data_point = data['w'].isel(time=t, zf=zlev, xh=xrange[ind], yh=yrange[ind]).values
#                 elif var=='qv':
#                     data_point = data['qv'].isel(time=t, zh=zlev, xh=xrange[ind], yh=yrange[ind]).values
#                 elif var=='qvconv':
#                     ##########################################################################################
#                     def get_3dtime_data(data,varname,t,zlev):
#                         try:
#                             cloud_var=data[varname].isel(time=t,zh=zlev).values
#                         except Exception as e:
#                             cloud_var=data[varname].isel(time=t,zf=zlev).values
#                         return cloud_var
#                     def cd2d_zlev(f,dx,dy): #size not compatible, cant calculate adjacent gradient
#                         ddx = (
#                                 f[:, 1:  ]
#                                 -
#                                 f[:, 0:-1]
#                             ) / (
#                             2 * dx
#                         )
                    
#                         ddy = (
#                             f[1:, :]
#                             -
#                             f[0:-1, :]
#                         ) / (
#                             2 * dy
#                         )
                    
#                         return ddx, ddy
                    
#                     #getting moisture convergence 
#                     dx=data['xf'][1]-data['xf'][0];dx=dx.item();dy=dx
#                     u=get_3dtime_data(data,'u',t,zlev)       
#                     v=get_3dtime_data(data,'v',t,zlev)  
    
#                     [dudx,dudy]=cd2d_zlev(u,dx,dy)
#                     [dvdx,dvdy]=cd2d_zlev(v,dx,dy)
#                     conv = -(dudx + dvdy)
                    
                    
#                     qv=get_3dtime_data(data,'qv',t,zlev)  
#                     data_point = qv*conv
#                     data_point=data_point[yrange[ind],xrange[ind]]
#                 lr[ind]=data_point
#                 ##########################################################################################
#             result=np.vstack((result,lr))
#     result=result[1:]
#     return result


# t=30
# result1=lrdata(t,'w')
# result2=lrdata(t,'qv')
# result3=lrdata(t,'qvconv')
# ds1= xr.Dataset({'resultw': (['x', 'y'], result1),
#                  'resultqv': (['x', 'y'], result2),
#                  'resultqvconv': (['x', 'y'], result3)})
# ds1.to_netcdf('leftright_savesingle.nc') 

In [ ]:
# # #Comparing Vertical Velocity and Moisture around CL for all timesteps TESTING
# zlev=3#
# whereCLmean=np.round(whereCL['maxconv_x'].isel(z=zlev).mean(dim='time'),0).astype(int)#
# datameanu=data['u'].isel(zh=zlev).mean(dim='time')#
# datameanv=data['v'].isel(zh=zlev).mean(dim='time')#
# datameanw=data['w'].isel(zf=zlev).mean(dim='time')#
# datameanqv=data['qv'].isel(zh=zlev).mean(dim='time')#
# # find left and right of CL axis 
# import math
# kms=np.argmax(data['xh'].values-data['xh'][0].values >= 1) #finds how many x grids is 1 km
# def mirror(lst):
#     lst_mirror=lst[::-1]
#     return lst_mirror
# def make_xy(n):
#     dx=data['xh'].values[1]-data['xh'][0].values;dy=dx
#     #x
#     lst0=[-n]*n
#     lst1=list(np.arange(-(2*n-1)/2,((2*n-1)/2)+1,1))
#     lst2=[n]*(2*n)
#     lst3=[-n]*n
#     x=lst0+lst1+lst2+mirror(lst1)+lst3
#     x=np.array([ind*dx for ind in x])
#     #y
#     y=[]
#     arr1 = list(np.arange(2*n, (8*n)))
#     arr2 = list(np.arange(0, (2*n-1)+1))
#     rearange = arr1+arr2
#     y=[x[ind] for ind in rearange]
#     y=np.array([ind*dy for ind in y])
#     return x, y
# def make_inds_xy(n):
#     #inds_x
#     lst0=[-n]*n
#     lst1=list(np.arange(-n+1,(n-1)+1,1))
#     lst2=[n]*(2*n+1)
#     lst3=[-n]*(n+1)
#     inds_x=lst0+lst1+lst2+mirror(lst1)+lst3
#     inds_x=np.array(inds_x)
#     #inds_y
#     arr1 = list(np.arange(2*n, (8*n)))
#     arr2 = list(np.arange(0, (2*n-1)+1))
#     rearange = arr1+arr2
#     inds_y=np.array([inds_x[ind] for ind in rearange])
#     return inds_x, inds_y

# def leftright(u,v): #TESTING
#     #(0) getting wind angle
#     theta=math.atan2(v,u) #returns wind angle between -180 and 180
#     # print(f'theta is {math.degrees(theta)} deg')
    
#     ns=2*kms #2 kms around CL
#     #(1) starting for loop
#     xs,ys=[],[]
#     dx=data['xh'].values[1]-data['xh'][0].values;dy=dx
#     for n in range(1,(ns)+1):
#         #(2) make x and y
#         [x,y]=make_xy(n)
#         #(3) make indx and indy
#         [inds_x,inds_y]=make_inds_xy(n)
#         #(4) find tau
#         tau = np.arctan2(y, x)
#         #(5) use search sorted to find which range theta falls in 
#         ind=(8*n-2)-(np.searchsorted(mirror(tau),theta)-1)
#         #(6) get x and y inds
#         if np.degrees(theta) in [-180,180]: #needed since atan2 function discontinuity
#             xs.append(-n);ys.append(0)
#         else:
#             xs.append(inds_x[ind]);ys.append(inds_y[ind]) 
#         # print([inds_x[ind],inds_y[ind]])
#     xs=[-ind for ind in mirror(xs)]+[0]+xs
#     ys=[-ind for ind in mirror(ys)]+[0]+ys
#     # print(xs,ys)
#     return xs,ys
# ############################################################################################################
# def lrdata(t,var):
#     t=t#
#     zlev=3
#     n=2*kms
#     result=np.zeros((2*n+1))
#     for ylev in range(len(data['yh'])):
#         print(f'y level: {ylev}')
#         CL=whereCLmean.isel(y=ylev).values
#         xlev=np.where(CL!=-1)[0]
#         for xlev in xlev:
#             u=datameanu.isel(yh=ylev,xf=xlev).values
#             v=datameanv.isel(yf=ylev ,xh=xlev).values
#             [xs,ys]=leftright(u,v);  
            
#             xrange=[ind+xlev for ind in xs]
#             yrange=[np.mod(ind+ylev,len(data['yh'])) for ind in ys]
    
#             n=2*kms
#             lr = np.zeros(2*n+1)
#             for ind in range(len(xrange)):
#                 if var=='w':
#                     data_point = datameanw.isel(xh=xrange[ind], yh=yrange[ind]).values
#                 elif var=='qv':
#                     data_point = datameanqv.isel(xh=xrange[ind], yh=yrange[ind]).values
#                 elif var=='qvconv':
#                     ##########################################################################################
#                     def get_3dtime_data(data,varname,t,zlev):
#                         t=t#
#                         zlev=zlev#
#                         varname=varname#
#                         try:
#                             cloud_var=data.values
#                         except Exception as e:
#                             cloud_var=data.values
#                         return cloud_var
#                     def cd2d_zlev(f,dx,dy): #size not compatible, cant calculate adjacent gradient
#                         ddx = (
#                                 f[:, 1:  ]
#                                 -
#                                 f[:, 0:-1]
#                             ) / (
#                             2 * dx
#                         )
                    
#                         ddy = (
#                             f[1:, :]
#                             -
#                             f[0:-1, :]
#                         ) / (
#                             2 * dy
#                         )
                    
#                         return ddx, ddy
                    
#                     #getting moisture convergence 
#                     dx=data['xf'][1]-data['xf'][0];dx=dx.item();dy=dx
#                     # u=get_3dtime_data(datamean,'u',t,zlev)  #    
#                     # v=get_3dtime_data(datamean,'v',t,zlev)  #
#                     u=datameanu.values
#                     v=datameanv.values
    
#                     [dudx,dudy]=cd2d_zlev(u,dx,dy)
#                     [dvdx,dvdy]=cd2d_zlev(v,dx,dy)
#                     conv = -(dudx + dvdy)
                    
                    
#                     # qv=get_3dtime_data(datamean,'qv',t,zlev)#
#                     qv=datameanqv
#                     data_point = qv*conv
#                     data_point=data_point[yrange[ind],xrange[ind]]
#                 lr[ind]=data_point
#                 ##########################################################################################
#             result=np.vstack((result,lr))
#     result=result[1:]
#     return result


# t=30#
# result1=lrdata(t,'w')
# result2=lrdata(t,'qv')
# result3=lrdata(t,'qvconv')
# ds1= xr.Dataset({'resultw': (['x', 'y'], result1),
#                  'resultqv': (['x', 'y'], result2),
#                  'resultqvconv': (['x', 'y'], result3)})
# ds1.to_netcdf('leftright_savemean.nc') 

In [ ]:
# #Contour Plots
# results=xr.open_dataset('leftright_save_single.nc')
# resultw=results['resultw']
# resultqv=results['resultqv']
# resultqvconv=results['resultqvconv']
# #zooming in plot
# #sectioned mean
# resultw=resultw.coarsen(x=100,boundary="pad").mean()
# resultqv=resultqv.coarsen(x=100,boundary="pad").mean()
# resultqvconv=resultqvconv.coarsen(x=100,boundary="pad").mean()
# # #running mean
# # resultw=resultw.rolling(x=100, center=True, min_periods=1).mean()
# # resultqv=resultqv.rolling(x=100, center=True, min_periods=1).mean()
# # resultqvconv=resultqvconv.rolling(x=100, center=True, min_periods=1).mean()

# kms=np.argmax(data['xh'].values-data['xh'][0].values >= 1) #finds how many x grids is 1 km
# def lrplot(result,cmap,label,var,vartitle):
#     zlev=3
#     if var in ['w','qvconv']:
#         norm = colors.TwoSlopeNorm(vcenter=0)
#         contour=plt.contourf(result,cmap=cmap,norm=norm,levels=10)
#     else:
#         contour=plt.contourf(result,cmap=cmap,levels=10)
    
#     colorbar=plt.colorbar(contour, pad=pad)
#     colorbar.set_label(label=label, fontsize=8)
#     ####################################################
#     plt.xticks(range(len(result[0]))); 
#     ticks, _ = plt.xticks()
#     newticks=(ticks-2*kms); newticks=newticks.astype(int)
#     plt.xticks(ticks=ticks, labels=newticks);
    
#     ticks, _ = plt.yticks()
#     newticks=ticks-np.floor(len(result)/2); newticks=newticks.astype(int)
#     plt.yticks(ticks=ticks, labels=newticks);
#     ####################################################
#     plt.axvline(x=2*kms, color='green',label='CL',linewidth=2) #plotting coast
#     ####################################################
#     plt.title(f'Horizontal Plot of {vartitle} around CL')
#     # plt.legend(loc='upper left',fontsize=6)
#     plt.xlabel('x (km)')
#     plt.ylabel('y (km)')
#     value=data['zf'][zlev].values; 
#     plt.text(2*kms*1.6,len(result)*0.85,f'z={value:.2f} m',fontsize=8) 
#     ####################################################

# pad=0;dpi=600;width=5
# channel_aspect_ratio = 1
# plt.figure(figsize=(width, width/channel_aspect_ratio),dpi=dpi);
# plt.subplot(3, 1, 1);plt.subplots_adjust(hspace=0.5)
# lrplot(resultw,'bwr',r'$m\ s^{-1}$','w','W')
# plt.subplot(3, 1, 2);
# lrplot(resultqv,'Blues',r'$kg kg^{-1}$','qv','QV')
# plt.subplot(3, 1, 3);
# lrplot(resultqvconv,'bwr',r'$kg\ kg^{-1}\ s^{-1}$','qvconv','QV Conv')
# plt.tight_layout()

# plt.savefig(os.path.join(folder_path, f"CLleftrightcontour.jpg"),dpi=dpi) 

In [ ]:
# #Scatter Plots
# n=2*kms
# lrange=np.arange(0,(n))
# mrange=np.arange(n,(n)+1,1)
# rrange=np.arange(n+1,(2*n)+1)

# pad=0;dpi=300;width=10
# channel_aspect_ratio=5
# plt.figure(figsize=(width, 20/channel_aspect_ratio),dpi=dpi);

# lresultw=resultw[:,lrange].mean(dim='y');mresultw=resultw[:,mrange].mean(dim='y');rresultw=resultw[:,rrange].mean(dim='y')
# plt.subplot(2, 2, 1); #plt.subplots_adjust(hspace=0.5)
# plt.scatter(np.arange(len(lresultw)),lresultw,label="w CL-2km",color='blue',s=20); 
# plt.scatter(np.arange(len(mresultw)),mresultw,label="w CL",color='green',s=20);
# plt.scatter(np.arange(len(rresultw)),rresultw,label="w CL+2km",color='orange',s=20);
# plt.axhline(np.mean(lresultw),color='blue')
# plt.axhline(np.mean(mresultw),color='green')
# plt.axhline(np.mean(rresultw),color='orange')
# plt.xlabel('time in 5 minute increments')
# plt.ylabel(r'$m\ s^{-1}$')
# plt.legend(loc='upper left',fontsize=10)
# plt.title('comparing mean W +-2km of CL location at each time')

# lresultqv=resultqv[:,lrange].mean(dim='y');mresultqv=resultqv[:,mrange].mean(dim='y');rresultqv=resultqv[:,rrange].mean(dim='y')
# plt.subplot(2,2,2)
# plt.scatter(np.arange(len(lresultqv)),lresultqv,label="qv CL-2km",color='blue',s=20); 
# plt.scatter(np.arange(len(mresultqv)),mresultqv,label="qv CL",color='green',s=20); 
# plt.scatter(np.arange(len(rresultqv)),rresultqv,label="qv CL+2km",color='orange',s=20); 
# plt.axhline(np.mean(lresultqv),color='blue')
# plt.axhline(np.mean(mresultqv),color='green')
# plt.axhline(np.mean(rresultqv),color='orange')
# plt.xlabel('time in 5 minute increments')
# plt.ylabel(r'$kg\ kg^{-1}$')
# plt.legend(loc='upper left',fontsize=10)

# lresultqvconv=resultqvconv[:,lrange].mean(dim='y');mresultqvconv=resultqvconv[:,mrange].mean(dim='y');rresultqvconv=resultqvconv[:,rrange].mean(dim='y')
# plt.subplot(2,2,3)
# plt.scatter(np.arange(len(lresultqvconv)),lresultqvconv,label="qv conv CL-2km",color='blue',s=20); 
# plt.scatter(np.arange(len(mresultqvconv)),mresultqvconv,label="qv conv CL",color='green',s=20); 
# plt.scatter(np.arange(len(rresultqvconv)),rresultqvconv,label="qv conv CL+2km",color='orange',s=20); 
# plt.axhline(np.mean(lresultqvconv),color='blue')
# plt.axhline(np.mean(mresultqvconv),color='green')
# plt.axhline(np.mean(rresultqvconv),color='orange')
# plt.xlabel('time in 5 minute increments')
# plt.ylabel(r'$kg\ kg^{-1}\ s^{-1}$')
# plt.legend(loc='upper left',fontsize=10)

# plt.tight_layout()
# plt.savefig(os.path.join(folder_path, f"CLleftrightscatter.jpg"),dpi=dpi) 

In [ ]:
# # find left and right of CL axis TESTING 
# import math
# kms=np.argmax(data['xh'].values-data['xh'][0].values >= 1) #finds how many x grids is 1 km
# kms=5
# # kms=1
# def mirror(lst):
#     lst_mirror=lst[::-1]
#     return lst_mirror
# def make_xy(n):
#     dx=data['xh'].values[1]-data['xh'][0].values;dy=dx
#     #x
#     lst0=[-n]*n
#     lst1=list(np.arange(-(2*n-1)/2,((2*n-1)/2)+1,1))
#     lst2=[n]*(2*n)
#     lst3=[-n]*n
#     x=lst0+lst1+lst2+mirror(lst1)+lst3
#     x=np.array([ind*dx for ind in x])
#     #y
#     y=[]
#     arr1 = list(np.arange(2*n, (8*n)))
#     arr2 = list(np.arange(0, (2*n-1)+1))
#     rearange = arr1+arr2
#     y=[x[ind] for ind in rearange]
#     y=np.array([ind*dy for ind in y])
#     return x, y
# def make_inds_xy(n):
#     #inds_x
#     lst0=[-n]*n
#     lst1=list(np.arange(-n+1,(n-1)+1,1))
#     lst2=[n]*(2*n+1)
#     lst3=[-n]*(n+1)
#     inds_x=lst0+lst1+lst2+mirror(lst1)+lst3
#     inds_x=np.array(inds_x)
#     #inds_y
#     arr1 = list(np.arange(2*n, (8*n)))
#     arr2 = list(np.arange(0, (2*n-1)+1))
#     rearange = arr1+arr2
#     inds_y=np.array([inds_x[ind] for ind in rearange])
#     return inds_x, inds_y

# def leftright(degrees): #TESTING
#     #(0) getting wind angle
#     # theta=math.atan2(v,u) #returns wind angle between -180 and 180
#     theta=np.radians(degrees)
#     # print(f'theta is {math.degrees(theta)} deg')
    
#     ns=2*kms #2 kms around CL
#     #(1) starting for loop
#     xs,ys=[],[]
#     dx=data['xh'].values[1]-data['xh'][0].values;dy=dx
#     for n in range(1,(ns)+1):
#         #(2) make x and y
#         [x,y]=make_xy(n)
#         #(3) make indx and indy
#         [inds_x,inds_y]=make_inds_xy(n)
#         #(4) find tau
#         tau = np.arctan2(y, x)
#         #(5) use search sorted to find which range theta falls in 
#         ind=(8*n-2)-(np.searchsorted(mirror(tau),theta)-1)
#         #(6) get x and y inds
#         if np.degrees(theta) in [-180,180]: #needed since atan2 function discontinuity
#             xs.append(-n);ys.append(0)
#         else:
#             xs.append(inds_x[ind]);ys.append(inds_y[ind]) 
#         # print([inds_x[ind],inds_y[ind]])
#     # xs=[-ind for ind in mirror(xs)]+[0]+xs
#     # ys=[-ind for ind in mirror(ys)]+[0]+ys
#     # print(xs,ys)
#     return xs,ys

# import matplotlib.patches as patches
# def plot_5x5_grid(angle_deg,xs,ys,fig,ax):
#     ns=2*kms
#     #make grid
#     angle_rad = np.radians(angle_deg)
#     # fig, ax = plt.subplots() #used in update
#     for i in range(1, 2*ns+1):
#         ax.plot([i, i], [0, 2*ns+1], 'k-')
#         ax.plot([0, 2*ns+1], [i, i], 'k-')

#     ax.set_xlim(0, 2*ns+1)
#     ax.set_ylim(0, 2*ns+1)
#     ax.set_xticks(range(2*ns+2))
#     ax.set_yticks(range(2*ns+2))
#     ax.tick_params(axis='both', which='both', length=0)
#     ax.tick_params(axis='both', which='both', length=0)
#     ax.set_xticklabels([])
#     ax.set_yticklabels([])

#     #vector
#     dx = ns * np.cos(angle_rad)
#     dy = ns * np.sin(angle_rad)
#     ax.quiver(ns+0.5, ns+0.5, dx, dy, angles='xy', scale_units='xy', scale=1, color='r',zorder=2,label=f'{angle_deg:.0f} °')

#     #fill
#     #center
#     rect=patches.Rectangle((0+ns, 0+ns), 1, 1, linewidth=1, edgecolor='none', facecolor='black',label='center')
#     ax.add_patch(rect)
#     #rightside
#     for ind in range(len(xs)):
#         rect=patches.Rectangle((xs[ind]+ns, ys[ind]+ns), 1, 1, linewidth=1, edgecolor='none', facecolor='orange')
#         ax.add_patch(rect)
#         if ind == 0:
#             rect.set_label('CL right')
#     #leftside
#     xs2=[-ind for ind in xs];ys2=[-ind for ind in ys]
#     for ind in range(len(xs2)):
#         rect=patches.Rectangle((xs2[ind]+ns, ys2[ind]+ns), 1, 1, linewidth=1, edgecolor='none', facecolor='blue')
#         ax.add_patch(rect)
#         if ind == 0:
#             rect.set_label('CL left')
#     ax.legend(loc='center')
    
# # #Plotting
# # amount=36; 
# # fig, ax = plt.subplots()
# # for theta in np.concatenate((np.arange(0, 181, amount), np.arange(-180 + amount, 0, amount))):
# #     [xs,ys]=leftright(theta)
# #     plot_5x5_grid(theta,xs,ys,fig,ax)
    
# # for theta in np.arange(-180+amount,0,amount):
# #     [xs,ys]=leftright(theta)
# #     plot_5x5_grid(theta,xs,ys,fig,ax)

# #Animation
# from matplotlib.animation import FuncAnimation
# amount=1
# angles=np.concatenate((np.arange(0, 181, amount), np.arange(-180 + amount, 0, amount)))

# fig, ax = plt.subplots()
# def update(theta):
#     ax.clear()
#     xs, ys = leftright(theta)
#     plot_5x5_grid(theta, xs, ys,fig,ax)

# ani = FuncAnimation(fig, update, frames=angles, repeat=False)
# ani.save('grid_animation.gif', writer='pillow', fps=10)

In [ ]:
# import matplotlib.colors as mcolors
# uvwind_colors = [
# (255, 255, 255),  
# (90,211,255),
# (0,155,231), 
# (0,102,251),  
# (0,153,0),
# (57,255,0),
# (250,255,0), 
# (248,205,13),  
# (255,153,0),
# (255,0,0),
# (205,0,0),  
# (169,0,154),
# (255,0,255),
# (200,162,212),
# (112,73,142)]
# uvwind_colors = [(r / 255, g / 255, b / 255) for r, g, b in uvwind_colors]
# uvwind_cmap = mcolors.ListedColormap(uvwind_colors)
# uvwind_cmap

# # bounds = np.linspace(alongcoast.min(), alongcoast.max(), len(uvwind_colors)-1)
# # left = np.linspace(alongcoast.min(), 0, int((len(uvwind_colors)-1)/2))
# # right = np.linspace(0, alongcoast.max(), int((len(uvwind_colors)-1)/2))
# # bounds=np.concatenate((left,right[1:]))
# # norm = mpl.colors.BoundaryNorm(bounds, uvwind_cmap.N, extend='both')
# # plt.contourf(alongcoast,cmap=uvwind_cmap,norm=norm,levels=15)